In [1]:
import sys
print(sys.executable)

E:\Program Files (x86)\Anaconda3\envs\dissertation\python.exe


In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import numpy as np



sys.path.append('E:\GitHubProjects\dissertation\Scripts')
import helperfn as hf

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [3]:
from gensim.models import KeyedVectors

In [4]:
embedding_model = KeyedVectors.load_word2vec_format(r'E:\GitHubProjects\dissertation\word2vec\GoogleNews-vectors-negative300.bin', binary=True)
# If we don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
embedding_model.init_sims(replace=True)

In [6]:
embedding_model.most_similar('bad', topn=5)

[('good', 0.7190051078796387),
 ('terrible', 0.6828611493110657),
 ('horrible', 0.6702598333358765),
 ('Bad', 0.6698919534683228),
 ('lousy', 0.6647640466690063)]

In [7]:
#Get Data
data = hf.merge_datasets(r'E:\GitHubProjects\dissertation\scraper\approved_datasets')

#Clean reivews (X)

# #Remove stop words
# X = X.apply(lambda x: hf.remove_punctuations(x))

#Tokenizing the text using NLTK
#Tokenize [TODO dont split apostrophe e.g. don't but students' ] 
# TODO ******. doesnt segment full stop 
# tokenizer = RegexpTokenizer(pattern='\w+|\$[\d\.]+|\S+')
# tokenizer = RegexpTokenizer(r'\w+')
# https://stackoverflow.com/questions/42056872/how-to-remove-in-strings-with-regexptokenizer
tokenizer = RegexpTokenizer(r'[\w\']+') #not seperating apostrophe


data['tokenized_text'] = data['review'].apply(lambda x: tokenizer.tokenize(x)) 

#lower case
data['tokenized_text'] = data['tokenized_text'].apply(lambda x: hf.lower_token(x))


In [10]:
pd.set_option('display.max_colwidth', -1)

data['tokenized_text']

NameError: name 'pd' is not defined

In [37]:
#counting number of unique words in  all reviews to get size of vocabulary
corpus = set()
for row in data['tokenized_text']:
    corpus.update(row)
vocab_size = len(corpus)

In [104]:
X, y = list(data['tokenized_text']), data['score']
# Split the data
X_train, X_test, y_train, y_test = train_test_split(data["tokenized_text"], y, test_size=0.33, shuffle= True)

In [105]:
#Word Embeddings
embedding_vecor_length = 300
embedding_matrix = np.zeros((vocab_size, embedding_vecor_length))

for i, word in enumerate(corpus):
    try:
        embedding_vector = embedding_model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        pass
        # embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25), embedding_vecor_length)

In [123]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers


model = Sequential()

# Embedding Layer, output dimension is 300 because word2vec 300d
embedding_layer = Embedding(vocab_size, embedding_vecor_length, weights=[embedding_matrix], input_length=max_review_length, trainable=True)

sequence_input = Input(shape=(max_review_length,))
embedded_sequences = embedding_layer(sequence_input)

convs = []
filter_sizes = [2, 3, 4, 5, 6]
for filter_size in filter_sizes:
    l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
    l_pool = GlobalMaxPooling1D()(l_conv)
    convs.append(l_pool)

l_merge = concatenate(convs, axis=1)
x = Dropout(0.1)(l_merge)  
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
preds = Dense(2, activation='sigmoid')(x)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 600)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 600, 300)     737100      input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (None, 599, 200)     120200      embedding_10[0][0]               
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 598, 200)     180200      embedding_10[0][0]               
____________________________________________________________________________________________

In [111]:
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=32)

ValueError: Error when checking input: expected input_3 to have shape (600,) but got array with shape (1,)

In [ ]:
model = Sequential()

# Embedding Layer, output dimension is 300 because word2vec 300d
embedding_layer = Embedding(vocab_size, embedding_vecor_length, weights=[embedding_matrix], input_length=max_review_length), trainable=True)

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)


model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Display summary of the model
print(model.summary())

# Fit the model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          shuffle=True,
          epochs=nb_epochs,
          validation_data=(X_test, Y_test),
          callbacks=[EarlyStopping(min_delta=0.00025, patience=2)])




# model_cnn = Sequential()
# e = Embedding(100000, 100, input_length=1000)
# model_cnn.add(e)
# model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
# model_cnn.add(GlobalMaxPooling1D())
# model_cnn.add(Dense(256, activation='relu'))
# model_cnn.add(Dense(1, activation='sigmoid'))
# model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model_cnn.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
# score,acc = model_cnn.evaluate(x_val_seq, y_validation, verbose = 2, batch_size = 32)
# print("score: %.2f" % (score))
# print("acc: %.2f" % (acc))


In [37]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

#Get Data
data = hf.merge_datasets(r'E:\GitHubProjects\dissertation\scraper\approved_datasets')
# data = hf.balance_dataset(data)

#Clean reivews (X)

# #Remove stop words
# X = X.apply(lambda x: hf.remove_punctuations(x))

#Tokenizing the text using NLTK
#Tokenize [TODO dont split apostrophe e.g. don't but students' ] 
# TODO ******. doesnt segment full stop 
# https://stackoverflow.com/questions/42056872/how-to-remove-in-strings-with-regexptokenizer
tokenizer = RegexpTokenizer(r'[\w\']+') #not seperating apostrophe


data['tokenized_text'] = data['review'].apply(lambda x: tokenizer.tokenize(x)) 

#remove stop words
data['tokenized_text'] = data['tokenized_text'].apply(lambda x: [item for item in x if item not in stop])

#lower case
data['tokenized_text'] = data['tokenized_text'].apply(lambda x: hf.lower_token(x))

In [38]:
# X, y = list(data['review']), list(data['score'])
X, y = data['tokenized_text'], list(data['score'])

In [40]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#vocab size?
NUM_WORDS = 10000
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [41]:
#finding the maximum number of words the longests sentence contains
# length = []
# # TODO X_train might need to be a list not dataframe ["aa", "bb"]
# for x in X_train:
#     length.append(len(x.split()))
# max(length)

#my more memory efficient version
# longest = 0
# for x in X_train:
#     if len(x.split()) > longest:
#         longest = len(x.split())

#VERSION FOR TOKENIZED ROWS
longest = 0
for x in X:
    if len(x) > longest:
        longest = len(x)


#padding sequences to have the same length
x_train_seq = pad_sequences(sequences, maxlen=longest+5)

In [36]:
sequences_val = tokenizer.texts_to_sequences(X_test)
x_val_seq = pad_sequences(sequences_val, maxlen=longest+5)

In [42]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(x_train_seq, y, test_size=0.33, shuffle=True)

In [43]:
#Word Embeddings
#length of vectors in embedding word2vec
embedding_vecor_length = 300
embedding_matrix = np.zeros((NUM_WORDS, embedding_vecor_length))

for word, i in tokenizer.word_index.items():
    if i >= NUM_WORDS:
        continue
    try:
        embedding_vector = embedding_model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        pass
        # embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25), embedding_vecor_length)

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import MaxPooling1D


In [48]:
model_cnn = Sequential()
e = Embedding(NUM_WORDS, 300, weights=[embedding_matrix], input_length=longest+5, trainable=False)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=7, batch_size=32, verbose=2)

Train on 1095 samples, validate on 540 samples
Epoch 1/7
 - 0s - loss: 0.5654 - accuracy: 0.7114 - val_loss: 0.4151 - val_accuracy: 0.8519
Epoch 2/7
 - 0s - loss: 0.2842 - accuracy: 0.9023 - val_loss: 0.2990 - val_accuracy: 0.8722
Epoch 3/7
 - 0s - loss: 0.1650 - accuracy: 0.9388 - val_loss: 0.2620 - val_accuracy: 0.8981
Epoch 4/7
 - 0s - loss: 0.0922 - accuracy: 0.9781 - val_loss: 0.2610 - val_accuracy: 0.9037
Epoch 5/7
 - 0s - loss: 0.0464 - accuracy: 0.9936 - val_loss: 0.2661 - val_accuracy: 0.9056
Epoch 6/7
 - 0s - loss: 0.0254 - accuracy: 0.9973 - val_loss: 0.3042 - val_accuracy: 0.9000
Epoch 7/7
 - 0s - loss: 0.0161 - accuracy: 0.9973 - val_loss: 0.2803 - val_accuracy: 0.9019


In [49]:
# tk_test = Tokenizer()
# tk_test.fit_on_texts(X_test)
# index_list = tk_test.texts_to_sequences(X_test)
# x_train_padded = pad_sequences(index_list, maxlen=longest+5)

score = model_cnn.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.28029762616863957 / Test accuracy: 0.9018518328666687


In [50]:
from sklearn.metrics import confusion_matrix

y_pred = (model_cnn.predict(X_test).ravel()>0.5)+0 # predict and get class (0 if pred < 0.5 else 1)
confusion_matrix(y_test, y_pred)

array([[147,  33],
       [ 20, 340]], dtype=int64)